# Query Data with R
This notebook shows how to connect to a FinSpace Managed kdb cluster using the [rkdb](https://github.com/KxSystems/rkdb) library.

## Setup
Update [env.r](env.r) with you environment's information (github has an empty example).

## References
- [rkdb](https://github.com/KxSystems/rkdb) library (github)
- [paws](https://cran.r-project.org/web/packages/paws/index.html) library (CRAN)  
- [nanotime](https://cran.r-project.org/web/packages/nanotime/index.html) library (CRAN)  

In [1]:
#' Test if package is already installed
#' 
#' @param mypkg package name to check
#' @returns TRUE: if package is installed, else FALSE
#'
is.installed <- function(mypkg) {
    is.element(mypkg, installed.packages()[,1])    
}

#' Install package if not installed
#' 
#' @param x package name to confirm installation
#' @returns NULL
#'
pkgTest <- function(x) {
    if ( !is.installed(x) ) {
        install.packages(x, dep=TRUE)
    } 
    return(NULL)
}

#' Get a connection to a FinSpace managed kdb cluster
#' 
#' @param environment_id FinSpace environment ID
#' @param cluster_name FinSpace cluster name to connect to
#' @param user_name  FinSpace user connecting to cluster 
#' @returns A connection to the requested cluster
#'
get_connection <- function(environment_id, cluster_name, user_name) {
    # create finspace service client (uses default credentials)
    finspace <- paws::finspace()
    
    # Call service to get information about the FinSpace user
    user_resp <- finspace$get_kx_user(
      environmentId = environment_id,
      userName = user_name
    )
    
    # get the connection string to the cluster for the finspace user, passing the userARN
    conn_str <- finspace$get_kx_connection_string(
        environmentId = environment_id, 
        userArn = user_resp$userArn, 
        clusterName = cluster_name
    )
    
    # Break into parts for connection function call
    split_df <- str_split(conn_str$signedConnectionString, ":")
    
    # split the connection string
    conn_parts <- split_df[[1]]
    
    # get out the bits
    host<-gsub('/', '', conn_parts[3])
    port<-conn_parts[4]
    user<-conn_parts[5]
    pass<-conn_parts[6]
    
    # username password are packed together for connection call
    userpass<-paste(user, pass, sep=":")

    return(open_connection(host, port, userpass))
}

In [2]:
# ensure needed packages are installed
for (p in c("paws", "nanotime", "stringr", "devtools") ) {
    pkgTest(p)
}

# rkdb is not on CRAN, installing different (uses devtools)
if ( !is.installed("rkdb") ) {
    devtools::install_github('KxSystems/rkdb')
} 

In [3]:
library(rkdb)
library(nanotime)
library(stringr)

# Source FinSpace environment information
source("env.r")

# Cluster to connect to, created by welcome notebook: welcome.ipynb
CLUSTER_NAME<-"HDB_welcomedb"

In [4]:
# connect to cluster
conn <- get_connection(ENV_ID, CLUSTER_NAME, KDB_USERNAME)

In [5]:
## Available Tables in FinSpace Cluster
execute(conn, 'tables[]')

[1] "example"

In [6]:
## Sample Records from Quote table

execute(conn, 'select from example where date=min date, i<5')

date,sym,time,number
<date>,<chr>,<nanotime>,<int64>
2024-11-16,aaa,2024-11-16T14:44:14.737+00:00,53231
2024-11-16,aaa,2024-11-16T14:44:14.737+00:00,153560
2024-11-16,aaa,2024-11-16T14:44:14.737+00:00,449428
2024-11-16,aaa,2024-11-16T14:44:14.737+00:00,631966
2024-11-16,aaa,2024-11-16T14:44:14.737+00:00,941566
